# Simple Stohastic Strategy

This is a simplest strategy based on stochastic oscilator, in this first qualifing task you need to replace standard implementation of stochastic oscilator with your own implementation. See the source cell number 3.

Before completing the assignment, we recommend you familiarize yourself with the theory ([en](https://www.investopedia.com/terms/s/stochasticoscillator.asp), [ru](https://www.metatrader5.com/ru/terminal/help/indicators/oscillators/so)).

### 1. Preparations
At first one needs to prepare the workspace - load data and libraries

In [1]:
import xarray as xr
import numpy as np
import pandas as pd

import qnt.data    as qndata
import qnt.stats   as qnstats
import qnt.graph   as qngraph
import qnt.stepper as qnstepper
import qnt.forward_looking as qnfl
import qnt.xr_talib as qntalib
import plotly.graph_objs as go

import matplotlib.pyplot as plt
import datetime as dt
import time

In [2]:
data = qndata.load_data(tail = dt.timedelta(days=10*365), dims = ("time", "field", "asset"), forward_order = True)

fetched chunk 1/14 1s
fetched chunk 2/14 1s
fetched chunk 3/14 2s
fetched chunk 4/14 3s
fetched chunk 5/14 3s
fetched chunk 6/14 4s
fetched chunk 7/14 4s
fetched chunk 8/14 5s
fetched chunk 9/14 6s
fetched chunk 10/14 6s
fetched chunk 11/14 7s
fetched chunk 12/14 8s
fetched chunk 13/14 8s
fetched chunk 14/14 9s
Data loaded 9s


Available data explanation is [here](https://quantnet.ai/learn/:availableData_Table).

### 2. Weights allocation

This is a stub, it uses a standard TALIB implementation. **You should implement your own function.**

In [3]:
def calc_stochastic_d(data, period, d_period):
    prices = data.sel(field=['close', 'high', 'low'])
    close = prices.sel(field='close')
    min_low = prices.sel(field='low').rolling(time=period).min()
    max_high = prices.sel(field='high').rolling(time=period).max()
    k = 100 * (close - min_low) / (max_high - min_low)
    d = k.rolling(time=d_period).sum() / d_period
    return d

Next, this is an example of assigning weights to a portfolio instruments based on stochastic D.

More theory about weights and capital allocation you can find [here](https://quantnet.ai/learn/:gettingStarted).

The simplest way is to use stochastic as trend indicator

In [4]:
D = calc_stochastic_d(data, 50, 10)

is_liquid = data.loc[:,"is_liquid",:]
output = is_liquid.where(D > 60).fillna(0)

output = output / output.sum('asset')

This is a test for your function. This cell should print 'Passed'.

In [5]:
# Checking
threshold_diff = 0.1
threshold_count = 0.1
treshold_duration = 60

start_time = time.time()
D_imp = calc_stochastic_d(data, 5, 3)
duration = time.time() - start_time

D_ref = qntalib.STOCHF(data, 5, 3).sel(field='fastd')

diff = D_ref.fillna(0) - D_imp.fillna(0)

count = xr.where(abs(diff) > threshold_diff, 1, 0).sum()

if count < threshold_count * len(data.time) * len(data.asset) and duration < treshold_duration:
    print('Passed')
else:
    print('Failed')

Passed


### 3. Perfomance estimation
Once we have constructed an algorithm we need to evaluate it. At first, we need to calculate statistic.

In [6]:
stat = qnstats.calc_stat(data, output, slippage_factor=0.05)
display(stat.to_pandas().tail())

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-06-30,2.202831,0.012469,0.177031,-0.097413,-0.262725,0.127648,0.022598,1.0,1004.0,0.121569,16.917727
2020-07-01,2.207070,0.001925,0.176922,-0.095676,-0.262725,0.152242,0.026935,1.0,1010.0,0.121597,16.913710
2020-07-02,2.213851,0.003072,0.176928,-0.092898,-0.262725,0.155185,0.027457,1.0,1010.0,0.121814,16.914922
2020-07-06,2.237356,0.010617,0.177031,-0.083267,-0.262725,0.176252,0.031202,1.0,1010.0,0.121807,16.912465
2020-07-07,2.226410,-0.004892,0.177032,-0.087752,-0.262725,0.156960,0.027787,1.0,1010.0,0.121526,16.910478


In [7]:
# show plot with profit and losses:
performance = stat.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)")

We use a set of [criteria](https://quantnet.ai/learn/:quality_criteria) to evaluate the performance. You can submit your algorithm if it passes all the [requirements](https://quantnet.ai/learn/:requirements). 

In this task you need only to implement the algorithm and estimate the performance. We do not require you to submit the algoritm, but you can try anyway)

In [8]:
# Actual sharpe on a relevant timeframe. 
# According to the rules the Sharpe ratio must be greater than 1:
display(stat[-1:].sel(field = ["sharpe_ratio"]).transpose().to_pandas())

time,2020-07-07
field,
sharpe_ratio,0.15696


In [9]:
# According to the rules the correlation with other strategies must be less than 90%:
qnstats.print_correlation(output, data)


WARNING! This strategy correlates with other strategies.
The number of systems with a larger Sharpe ratio and correlation larger than 0.8: 1
The max correlation value (with systems with a larger Sharpe ratio): 0.9057601361340218
Current sharpe ratio(3y): 0.15696006723617797



### 4. Submit
If you are satisfied enough with your algorithm and it passes all the requirements you can submit it.

In [10]:
# Finally, write the output, it is necessary for submit.
qnstepper.write_output(output)

write output: /root/fractions.nc.gz


# 5. What is next?

Your next task on the hackathon will be to develop your own trading strategy and pass filters. So, it makes sense to pay your attention to example strategies in templates. You can start implementing your strategy now. On the hackathon, we will provide you all the necessary support.